In [ ]:
import os
import yfinance as yf
import quantstats as qs
import scipy.cluster.hierarchy as sch
import cvxopt as opt
import matplotlib.pyplot as plt
import shutil
import base64
import cvxpy as cp

from scipy.stats import spearmanr
from io import BytesIO
from datetime import timedelta
from IPython.display import Image
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold
from cvxopt import blas, solvers
from IPython.display import display
from scipy.optimize import minimize
from matplotlib.backends.backend_svg import FigureCanvasSVG

from class_model.model_prep import ModelPrep
from core.system import *
from core.operation import *

import warnings

warnings.filterwarnings('ignore')

# Functions

In [ ]:
#Reads in all the results in the "modelName" folder
def read_result(model_name, result_name):
    data = []
    result_data_dir = get_ml_result_model(live, model_name)
    for i, folder_name in enumerate(os.listdir(result_data_dir)):
        try:
            if folder_name.startswith("params"):
                folder_path = os.path.join(result_data_dir, folder_name)
                file_path = os.path.join(folder_path, f"{result_name}.parquet.brotli")
                print(os.path.basename(folder_path))
                data.append(pd.read_parquet(file_path))
        except:
            continue
    return pd.concat(data, axis=0).reset_index(drop=True)

#Calculates the product of the daily_ic_mean and maximum overall IC in each result to find the best performing model
def get_max_ic(data):
    collection={}
    for index, row in data.iterrows():
        collection[max(row.loc[(row.index.str.startswith("daily_metric"))])]=index
    max_ic_idx=collection[max(list(collection.keys()))]
    return data.iloc[max_ic_idx]

#Gets the files of the best performing model
def get_max_ic_file(data, model_name):
    files = {}
    time_index = data.to_frame().index.get_loc('time')
    param_vals = data.iloc[:time_index].values
    key = [f'{float(p)}' for p in (param_vals)]
    key = '_'.join(key)
    
    result_data_dir = get_ml_result_model(live, model_name) / f'params_{key}'
    for file in os.listdir(result_data_dir):
        if file.endswith(".parquet.brotli"):
            files[extract_first_string(file)]=pd.read_parquet(os.path.join(result_data_dir, file))
        elif file.endswith(".png"):
            img = os.path.join(result_data_dir, file)
            files[extract_first_string(file)]=Image(img)
    return files

def get_all(model_name):
    return get_max_ic_file(get_max_ic(read_result(model_name, 'metrics')), model_name)

def sign_accuracy(predictions, actual, target_sign, pred):
    accuracies = []  # To store accuracies for each ticker
    
    # Iterate through each ticker and calculate accuracy
    for ticker in predictions.index.levels[0]:
        ticker_group = predictions.loc[ticker]
        actual_group = actual.loc[ticker]
        
        # Determine if each pair has the same sign
        if pred == 'price':
            correct_signs = (np.sign(ticker_group) == np.sign(actual_group))
        elif pred == 'sign':
            ticker_group = np.where(ticker_group > 0.5, 1, -1)
            correct_signs = (ticker_group == np.sign(actual_group))
            
        # Filter by target sign if specified
        if target_sign == 'positive':
            mask = (np.sign(actual_group) == 1)
            correct_signs = correct_signs[mask]
        elif target_sign == 'negative':
            mask = (np.sign(actual_group) == -1)
            correct_signs = correct_signs[mask]
        
        # Calculate the accuracy and store it
        accuracy = np.mean(correct_signs) * 100  # Convert to percentage
        accuracies.append(accuracy)
        
    # Calculate and return the mean accuracy across all tickers
    mean_accuracy = np.nanmean(accuracies)
    return mean_accuracy

In [ ]:
def price(best_model_params, dir_path, iteration, plot):
    #Gets the predictions of the highest overall IC in the boosted round cases
    if iteration == False:
        best_prediction=best_model_params['predictions'][[str(extract_number(best_model_params['metrics'].loc[:, best_model_params['metrics'].columns.str.startswith("daily_metric")].idxmax(axis=1)[0])), 'i']]
    else:
        best_prediction=best_model_params['predictions'][[str(iteration), 'i']]
    actual_return=best_model_params['returns']
    #Merge actual returns and prediction returns
    merged = pd.merge(best_prediction, actual_return, left_index=True, right_index=True, how='left')
    merged.columns =['predictions', 'window', 'returns']
    merged.window = merged.window.astype(int)
    #Shift actual returns 1 day back
    merged['returns'] = merged.groupby('permno')['returns'].shift(-1)
    merged = merged.dropna()

    if plot == False:
        return merged
        
    print('Best num_iterations: ' + str(best_prediction.columns[0]))
    print(f"Neutral Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, None, 'price'), 2)}%")
    print(f"Positive Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, 'positive', 'price'), 2)}%")
    print(f"Negative Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, 'negative', 'price'), 2)}%")
    display(best_model_params['metrics'])
    
    # Convert to HTML
    df_html = best_model_params['metrics'].to_html(classes='my-table')

    # Prepare the plot
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10, 6))
    best_model_params['daily_metric'][best_prediction.columns[0]].rolling(window=42).mean().plot(
        ax=ax, linewidth=0.5, color='blue', linestyle='-', title='Daily Metric Plot'
    )
    ax.set(xlabel='Date', ylabel='Daily Metric')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()

    # Convert plot to SVG
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_base64 = base64.b64encode(img_stream.getvalue()).decode()

    with open(dir_path / 'metric.html', 'w') as f:
        f.write('''<!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <title>Metrics Report</title>
        <style>
            body {
                font-family: 'Times New Roman', serif;
                margin: 0;
                padding: 0;
                background-color: #ffffff;
                text-align: center;
                color: #000000;
            }
            header {
                background: #d3d3d3;
                color: #000000;
                text-align: center;
                padding-top: 30px;
                min-height: 70px;
                margin: 0 auto;
            }
            .container {
                width: 70%;
                margin: auto;
            }
            .main-content {
                padding: 30px;
            }
            .my-table {
                width: 100%; 
                border-collapse: collapse;
                font-size: 12px;
                table-layout: fixed; 
            }
            th, td {
                border: 2px solid #e3e3e3;
                padding: 10px;
                text-align: left;
                word-wrap: break-word;
            }
        </style>
        </head>
        <body>
            <header>
                <h1>Metrics Report</h1>
            </header>
            <div class="container">
                <div class="main-content">
                    <p>Best num_iterations: ''' + str(best_prediction.columns[0]) + '''</p>
                    <p>Neutral Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, None, 'price'), 2)}%" + '''</p>
                    <p>Positive Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, 'positive', 'price'), 2)}%" + '''</p>
                    <p>Negative Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, 'negative', 'price'), 2)}%" + '''</p>
                    ''' + df_html + '''
                    <img src="data:image/png;base64,''' + img_base64 + '''" alt="plot" />
                </div>
            </div>
        </body>
        </html>''')
    return merged

def plot_ensemble(merged, ic_by_day):
    print(f'Daily IC Mean: {round(ic_by_day.mean()[0], 5)}')
    # Prepare the plot
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10, 6))
    ic_by_day.rolling(window=42).mean().plot(ax=ax, linewidth=0.5, color='blue', linestyle='-', title='Daily Metric Plot')
    ax.set(xlabel='Date', ylabel='Daily Metric')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    
    # Convert plot to SVG
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_base64 = base64.b64encode(img_stream.getvalue()).decode()
    
    with open(dir_path / 'metric.html', 'w') as f:
        f.write('''<!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <title>Metrics Report</title>
        <style>
            body {
                font-family: 'Times New Roman', serif;
                margin: 0;
                padding: 0;
                background-color: #ffffff;
                text-align: center;
                color: #000000;
            }
            header {
                background: #d3d3d3;
                color: #000000;
                text-align: center;
                padding-top: 30px;
                min-height: 70px;
                margin: 0 auto;
            }
            .container {
                display: flex;
                justify-content: center;
                align-items: center;
                flex-direction: column;
            }
            .main-content {
                padding: 30px;
            }
            .my-table {
                width: 100%; 
                border-collapse: collapse;
                font-size: 12px;
                table-layout: fixed; 
            }
            th, td {
                border: 2px solid #e3e3e3;
                padding: 10px;
                text-align: left;
                word-wrap: break-word;
            }
        </style>
        </head>
        <body>
            <header>
                <h1>Metrics Report</h1>
            </header>
            <div class="container">
                <div class="main-content">
                    <p>Neutral Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, None, 'price'), 2)}%" + '''</p>
                    <p>Positive Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, 'positive', 'price'), 2)}%" + '''</p>
                    <p>Negative Accuracy: ''' + f"{round(sign_accuracy(merged.predictions, merged.returns, 'negative', 'price'), 2)}%" + '''</p>
                    <img src="data:image/png;base64,''' + img_base64 + '''" alt="plot" />
                </div>
            </div>
        </body>
        </html>''')

def sign(best_model_params):
    #Gets the predictions of the highest overall IC in the boosted round cases
    best_prediction=best_model_params['predictions'][str(extract_number(best_model_params['metrics'].loc[:, best_model_params['metrics'].columns.str.startswith("daily_metric")].idxmax(axis=1)[0]))].to_frame()
    actual_return=best_model_params['returns']
    #Merge actual returns and prediction returns
    merged = pd.merge(best_prediction, actual_return, left_index=True, right_index=True, how='left')
    merged.columns =['predictions', 'returns']
    #Shift actual returns 1 day back
    merged['returns'] = merged.groupby('permno')['returns'].shift(-1)
    merged = merged.dropna()
    print('Best num_iterations: ' + str(best_prediction.columns[0]))
    print(f"Neutral Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, None, 'sign'), 2)}%")
    print(f"Positive Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, 'positive', 'sign'), 2)}%")
    print(f"Negative Accuracy: {round(sign_accuracy(merged.predictions, merged.returns, 'negative', 'sign'), 2)}%")
    display(best_model_params['metrics'])
    return merged

def gain(gain):
    # Sort by mean gain in descending order
    sorted_df = gain.sort_values(by='mean', ascending=False)

    # Plot
    plt.figure(figsize=(15, 50))
    sorted_df['mean'].plot(kind='barh')
    plt.xlabel('Average Gain')
    plt.ylabel('Features')
    plt.title('Feature Importances based on Gain')
    plt.gca().invert_yaxis()  # to have the most important feature on top
    plt.tight_layout()
    plt.show()
    
def split(split):
    # Sort by mean gain in descending order
    sorted_df = split.sort_values(by='mean', ascending=False)

    # Plot
    plt.figure(figsize=(15, 50))
    sorted_df['mean'].plot(kind='barh')
    plt.xlabel('Average Split')
    plt.ylabel('Features')
    plt.title('Feature Importances based on Split')
    plt.gca().invert_yaxis()  # to have the most important feature on top
    plt.tight_layout()
    plt.show()

In [ ]:
def hrp(returns, stocks, date):
    def correl_dist(corr):
        # A distance matrix based on correlation, where 0<=d[i,j]<=1
        # This is a proper distance metric
        dist = ((1 - corr) / 2.)**.5  # distance matrix
        dist = squareform(dist)
        return dist

    def get_quasi_diag(link):
        # Sort clustered items by distance
        link = link.astype(int)
        sortIx = pd.Series([link[-1, 0], link[-1, 1]])
        numItems = link[-1, 3]  # number of original items
        while sortIx.max() >= numItems:
            sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
            df0 = sortIx[sortIx >= numItems]  # find clusters
            i = df0.index
            j = df0.values - numItems
            sortIx[i] = link[j, 0]  # item 1
            df1 = pd.Series(link[j, 1], index=(i + 1).tolist())
            sortIx = pd.concat([sortIx, df1]).sort_index()
            sortIx.index = range(sortIx.shape[0])
        return sortIx.tolist()

    def get_cluster_var(cov, cItems):
        # Compute variance per cluster
        cov_= cov.loc[cItems,cItems] # matrix slice
        # Compute the inverse_variance portfolio
        ivp = 1. / np.diag(cov_)
        ivp /= ivp.sum()
        w_= ivp.reshape(-1,1)
        cVar = np.dot(np.dot(w_.T,cov_),w_)[0,0]
        return cVar

    def get_rec_bipart(cov, sortIx):
        # Compute HRP alloc
        w = pd.Series(1, index=sortIx)
        cItems = [sortIx]  # initialize all items in one cluster
        while len(cItems) > 0:
            cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]  # bi-section
            for i in range(0, len(cItems), 2):  # parse in pairs
                cItems0 = cItems[i]  # cluster 1
                cItems1 = cItems[i + 1]  # cluster 2
                cVar0 = get_cluster_var(cov, cItems0)
                cVar1 = get_cluster_var(cov, cItems1)
                alpha = 1 - cVar0 / (cVar0 + cVar1)
                w[cItems0] *= alpha  # weight 1
                w[cItems1] *= 1 - alpha  # weight 2
        return w
    
    cov, corr = returns.cov(), returns.corr()
    # Construct a hierarchical portfolio
    dist = correl_dist(corr)
    link = sch.linkage(dist, 'single')
    sortIx = get_quasi_diag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = get_rec_bipart(cov, sortIx)
    return hrp.sort_index().values

def ivp(returns, stocks, date):
    # Compute the inverse-variance portfolio
    cov = returns.cov()
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp

def ewp(returns, stocks, date, leverage):
    weight = leverage/len(returns.columns)
    return np.full(len(returns.columns), weight)

def mvp(returns, stocks, date):
    cov = returns.cov()
    cov = cov.T.values
    n = len(cov)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

    # Convert to cvxopt matrices
    S = opt.matrix(cov)
    #pbar = opt.matrix(np.mean(returns, axis=1))
    pbar = opt.matrix(np.ones(cov.shape[0]))

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    solvers.options['show_progress'] = False
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x'] for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER    
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO    
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(list(wt))

# Minimize Factor Loading Optimization
def mfl(data, type, sb, factor_weights):
    def objective_cvxpy(weights, betas, factor_weights):
        # Initialize the alternative objective
        alternative_objective = 0.0
    
        # Loop through each factor and calculate its contribution to the objective
        for factor, weight in factor_weights.items():
            port_beta = cp.matmul(weights, betas[factor])
            alternative_objective += weight * cp.square(port_beta)
    
        return alternative_objective
        
    collect_weight = []
    count = 0 
    print(f'Processing {type}...')
    for date, row in data.iterrows():
        # Get betas for given stocks at day t
        betas = sb.loc[sb.index.get_level_values(1) == date]
        stocks = row[type]
        betas = get_stocks_data(betas, stocks)
        betas = betas.fillna(0)

        # CVXPY setup
        weights = cp.Variable(betas.shape[0])
        objective = cp.Minimize(objective_cvxpy(weights, betas, factor_weights))
        constraints = [cp.sum(weights) == 1, weights >= 0, weights <= 1]
        prob = cp.Problem(objective, constraints)
        
        # Solve the problem
        # prob.solve()
        prob.solve(solver='ECOS')
        collect_weight.append(weights.value)
        
        # Track performance
        if count % 30 == 0:
            print(date)
        count += 1
    
    return collect_weight

In [ ]:
def process_period(period, period_returns, num_stocks, candidates, threshold):
    # Find sp500 candidates for the given year and assign it to data
    period_year = period.index.get_level_values('date')[0].year
    sp500 = candidates[period_year]
    tickers = common_stocks(sp500, period)
    sp500_period = get_stocks_data(period, tickers)
    print("----------------------------------------------")
    print(period_year)
    print('Num of tickers: ' + str(len(tickers)))

    # Filter the DataFrame to only include rows with market cap over the threshold
    filtered_period = period[period['market_cap'] > threshold]
        
    # Group by date and compute long and short stocks and their returns
    for date, stocks in filtered_period.groupby('date'):
        sorted_stocks = stocks.sort_values(by='predictions')
        long_stocks = sorted_stocks.index.get_level_values('ticker')[-num_stocks:]
        short_stocks = sorted_stocks.index.get_level_values('ticker')[:num_stocks]
        
        # Store results in period_returns DataFrame
        period_returns.loc[date] = [long_stocks.tolist(), sorted_stocks.iloc[-num_stocks:].returns.values,
                                    short_stocks.tolist(), sorted_stocks.iloc[:num_stocks].returns.values]

def backtest(data, num_stocks, threshold):
    global live
    # Set portfolio weights and other tracking variables
    period_returns = pd.DataFrame(columns=['longStocks', 'longRet', 'shortStocks', 'shortRet'])
    
    # Get candidates
    candidates = get_candidate(live)
    
    # Loop over each group in tic.groupby('window')
    for _, df in tic.groupby('window'):
        df = df.reset_index().set_index(['ticker', 'date']).drop('window', axis=1)
        process_period(df, period_returns, num_stocks, candidates, threshold)
    
    return period_returns

def plot_hist(data, date):
    pred = data.predictions.loc[data.index.get_level_values('date')==date]
    ret = data.returns.loc[data.index.get_level_values('date')==date]
    plt.hist(pred, bins='auto', edgecolor='black', alpha=0.5, label=f"Pred: {date}")
    plt.hist(ret, bins='auto', edgecolor='red', alpha=0.5, label=f"Real: {date}")
    plt.title('Histogram of values')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
def sharpe_ret(best_model_params, iteration):
    #Gets the predictions of the highest overall IC in the boosted round cases
    if iteration == False:
        best_prediction=best_model_params['predictions'][[str(extract_number(best_model_params['metrics'].loc[:, best_model_params['metrics'].columns.str.startswith("daily_metric")].idxmax(axis=1)[0])), 'i']]
    else:
        best_prediction=best_model_params['predictions'][[str(iteration), 'i']]
    actual_return=best_model_params['returns']
    #Merge actual returns and prediction returns
    merged = pd.merge(best_prediction, actual_return, left_index=True, right_index=True, how='left')
    merged.columns =['predictions', 'window', 'returns']
    merged.window = merged.window.astype(int)
    #Shift actual returns 1 day back
    merged['returns'] = merged.groupby('permno')['returns'].shift(-1)
    merged = merged.dropna()
    return merged

def sharpe_process_period(period, period_returns, num_stocks, candidates, threshold):
    # Find sp500 candidates for the given year and assign it to data
    period_year = period.index.get_level_values('date')[0].year
    # sp500 = candidates[period_year]
    # tickers = common_stocks(sp500, period)
    # sp500_period = get_stocks_data(period, tickers)

    # Filter the DataFrame to only include rows with market cap over the threshold
    filtered_period = period[period['market_cap'] > threshold]

    # Group by date and compute long and short stocks and their returns
    for date, stocks in filtered_period.groupby('date'):
        sorted_stocks = stocks.sort_values(by='predictions')
        long_stocks = sorted_stocks.index.get_level_values('permno')[-num_stocks:]
        short_stocks = sorted_stocks.index.get_level_values('permno')[:num_stocks]
        
        # Store results in period_returns DataFrame
        period_returns.loc[date] = [long_stocks.tolist(), sorted_stocks.iloc[-num_stocks:].returns.values,
                                    short_stocks.tolist(), sorted_stocks.iloc[:num_stocks].returns.values]


def sharpe_backtest(data, num_stocks, threshold):
    global live
    # Set portfolio weights and other tracking variables
    period_returns = pd.DataFrame(columns=['longStocks', 'longRet', 'shortStocks', 'shortRet'])
    
    # Get candidates
    candidates = get_candidate(live)
    
    # Loop over each group in tic.groupby('window')
    for _, df in tic.groupby('window'):
        df = df.reset_index().set_index(['permno', 'date']).drop('window', axis=1)
        sharpe_process_period(df, period_returns, num_stocks, candidates, threshold)
    
    return period_returns

In [ ]:
def port_opt(data, window_size, port_opt_func, option, leverage, file=None, factor_weights=None, factor_cols=None):
    global live
    assert port_opt_func==ewp or port_opt_func==hrp or port_opt_func==mvp or port_opt_func==ivp or port_opt_func==mfl, 'Must use callable function ewp, hrp, mvp, ivp, or mfl'
    if port_opt_func==hrp or port_opt_func==mvp or port_opt_func==ivp or port_opt_func==mfl:
        assert window_size>0, 'Window size must be greater than 0 if using hrp, mvp, ivp, or mfl'
    @ray.remote
    def exec_port_opt(i, data, window_size, port_opt_func, option):
        if option == 'long':  
            # Get window data for long returns
            long_returns = pd.DataFrame(data['longRet'][i:i+window_size].tolist())
            long_stocks = data['longStocks'][i:i+window_size][-1]
            date = data.iloc[i:i+window_size].index[-1]
            # Perform desired portfolio optimization
            long_weights = port_opt_func(long_returns, long_stocks, date)
            total_ret = np.sum(long_returns.iloc[-1] * long_weights)
        elif option == 'short':
            # Get window data for short returns
            short_returns = -1 * pd.DataFrame(data['shortRet'][i:i+window_size].tolist())
            short_stocks = data['shortStocks'][i:i+window_size][-1]
            date = data.iloc[i:i+window_size].index[-1]
            # Perform desired portfolio optimization
            short_weights = port_opt_func(short_returns, short_stocks, date)
            total_ret = np.sum(short_returns.iloc[-1] * short_weights)
        elif option == 'both':
            # Get window data for long returns and short returns
            # Perform desired portfolio optimization
            date = data.iloc[i:i+window_size].index[-1]
            long_returns = pd.DataFrame(data['longRet'][i:i+window_size].tolist())
            long_stocks = data['longStocks'][i:i+window_size][-1]
            long_weights = port_opt_func(long_returns, long_stocks, date)
            
            short_returns = -1 * pd.DataFrame(data['shortRet'][i:i+window_size].tolist())
            short_stocks = data['shortStocks'][i:i+window_size][-1]
            short_weights = port_opt_func(short_returns, short_stocks, date)
            total_ret = np.sum(long_returns.iloc[-1] * long_weights) + np.sum(short_returns.iloc[-1] * short_weights)
            
        return total_ret
    
    if port_opt_func == ewp:
        if option == 'long':
            # Get long returns
            long_returns = pd.DataFrame(data['longRet'].tolist())
            long_weights = port_opt_func(long_returns, None, None, leverage)

            # Calculate equal-weights and sum up to get total strategy return
            long_weights = np.tile(long_weights, (len(long_returns), 1))
            total_ret = np.sum(long_returns.values * long_weights, axis=1)
        elif option == 'short':
            # Get short returns
            short_returns = -1 *pd.DataFrame(data['shortRet'].tolist())
            short_weights = port_opt_func(short_returns, None, None, leverage)

            # Calculate equal-weights and sum up to get total strategy return
            short_weights = np.tile(short_weights, (len(short_returns), 1))
            total_ret = np.sum(short_returns.values * short_weights, axis=1)
        elif option == 'both':
            # Get long returns
            long_returns = pd.DataFrame(data['longRet'].tolist())
            long_weights = port_opt_func(long_returns, None, None, leverage)
            long_weights = np.tile(long_weights, (len(long_returns), 1))

            # Get short returns
            short_returns = -1 * pd.DataFrame(data['shortRet'].tolist())
            short_weights = port_opt_func(short_returns, None, None, leverage)
            short_weights = np.tile(short_weights, (len(short_returns), 1))

            # Calculate equal-weights and sum up to get total strategy return
            total_ret = np.sum(long_returns.values * long_weights, axis=1) + np.sum(short_returns.values * short_weights, axis=1)
        data['totalRet'] = total_ret
        return data

    if port_opt_func == mfl:
        # Read in calculate betas and convert from permno to ticker index
        stock = read_stock(get_load_data_large_dir() / 'permno_to_train_fund.csv')
        sb = ModelPrep(factor_name=file, group='permno', interval='D', kind='price', stock=stock, div=False, start='2006-01-01', end='2023-01-01', save=False).prep()
        ticker = pd.read_parquet(get_parquet_dir(live) / 'data_ticker.parquet.brotli')
        sb = sb.merge(ticker, left_index=True, right_index=True, how='left')
        sb = sb.reset_index().set_index(['ticker', 'date'])
        sb = sb.sort_index(level=['ticker', 'date'])
        sb = sb.drop('permno', axis=1)
        sb = sb[factor_cols]
        # For some reason, reseting index and setting it to ticker, date creates duplicate indices (should probably check this out)
        sb = sb[~sb.index.duplicated(keep='first')]
        
        if option == 'long':
            # Calculate minimized-factor loading weights and sum it up to get strategy return
            long_weights = mfl(data, 'longStocks', sb, factor_weights)
            long_returns = pd.DataFrame(data['longRet'].tolist())
            long_weights_df = pd.DataFrame(long_weights)
            total_ret = np.sum(long_returns.values * long_weights_df.values, axis=1)
        elif option == 'short':
            # Calculate minimized-factor loading weights and sum it up to get strategy return
            short_weights = mfl(data, 'shortStocks', sb, factor_weights)
            short_returns = -1 *pd.DataFrame(data['shortRet'].tolist())
            short_weights_df = pd.DataFrame(short_weights)
            total_ret = np.sum(short_returns.values * short_weights_df.values, axis=1)
        elif option == 'both':
            # Get long returns
            long_weights = mfl(data, 'longStocks', sb, factor_weights)
            long_returns = pd.DataFrame(data['longRet'].tolist())

            # Get short returns
            short_weights = mfl(data, 'shortStocks', sb, factor_weights)
            short_returns = -1 *pd.DataFrame(data['shortRet'].tolist())

            # Calculate minimized-factor loading weights and sum it up to get strategy return
            long_weights_df = pd.DataFrame(long_weights)
            short_weights_df = pd.DataFrame(short_weights)
            total_ret = np.sum(long_returns.values * long_weights_df.values, axis=1) + np.sum(short_returns.values * short_weights_df.values, axis=1)
        data['totalRet'] = total_ret
        return data
        
    ray.init(num_cpus=16, ignore_reinit_error=True)
    start_time = time.time()
    total_ret_collect = ray.get([exec_port_opt.remote(i, data, window_size, port_opt_func, option) for i in range(0, len(data) - window_size + 1)])
    end_time = time.time()
    elapsed_time = end_time - start_time
    rounded_time = round(elapsed_time, 2)
    print(f"Elapsed time: {rounded_time} seconds")
    ray.shutdown()
    
    data = data[window_size-1:]
    data['totalRet'] = total_ret_collect
    return data 

# Read in Results

In [ ]:
live = False
trial = 'lightGBM_trial_231'
files = read_result(trial, 'metrics')

In [ ]:
live = True
format_end = date.today().strftime('%Y%m%d')
model_name = f'lightgbm_{format_end}'
files = read_result(model_name, 'metrics')

In [ ]:
keep = {}
ticker = pd.read_parquet(get_parquet(live) / 'data_ticker.parquet.brotli')
misc = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
for i, row in files.iterrows():
    read_file = get_max_ic_file(row, trial)
    returns = sharpe_ret(read_file, iteration=False)
    # Convert Permno to Ticker
    tic = returns.merge(ticker, left_index=True, right_index=True, how='left')
    tic = tic.merge(misc, left_index=True, right_index=True, how='left')
    tic = tic.reset_index().set_index(['window', 'permno', 'date'])
    pred = sharpe_backtest(tic, num_stocks=20, threshold=10_000_000_000)
    equal_weight = port_opt(data=pred, window_size=21, port_opt_func=ewp, option='both', leverage=0.5)
    short_weight = port_opt(data=pred, window_size=21, port_opt_func=ewp, option='short', leverage=1)['totalRet']
    long_weight = port_opt(data=pred, window_size=21, port_opt_func=ewp, option='long', leverage=1)['totalRet']
    stock = equal_weight['totalRet']
    sharpe = qs.stats.sharpe(stock)    
    calmar = qs.stats.calmar(stock)
    print('-' * 60)
    print(f'Row: {i}')
    display(read_file['metrics'])
    print(f'SHARPE Ratio: {sharpe}')
    print(f'CALMAR Ratio: {calmar}')
    keep[i] = sharpe

//|

In [ ]:
qs.reports.full(long_weight, 'SPY')

# Minimizing Market Exposure

In [ ]:
def mfl(data, stocks_to_use, col_name, exp, min_exp_weights, lambda_reg):
    def objective_cvxpy(weights, betas, min_exp_weights, lambda_reg):
        # Initialize the alternative objective
        factor_exposure  = 0.0
        # Loop through each factor and calculate its contribution to the objective
        for factor, weight in min_exp_weights.items():
            port_beta = cp.matmul(weights, betas[factor])
            factor_exposure += weight * cp.square(port_beta)
            
        l2_reg = cp.norm(weights, 2)**2
        return factor_exposure + lambda_reg * l2_reg

    collect_weight = []
    df_collect = []
    count = 0 
    print(f'Processing {type}...')
    for date, row in stocks_to_use.iterrows():
        stocks = stocks_to_use.loc[stocks_to_use.index == date][col_name][0]
        # Get betas for given stocks at day t
        betas = exp.loc[exp.index.get_level_values('date') == date]
        betas = get_stocks_data(betas, stocks)
        betas = betas.sort_index(level=['permno', 'date'])
        betas = betas.fillna(0)

        # CVXPY setup
        weights = cp.Variable(betas.shape[0])
        objective = cp.Minimize(objective_cvxpy(weights, betas, min_exp_weights, lambda_reg))
        constraints = [
            cp.sum(weights) == 1,
            weights >= 0.001,
            weights <= 0.10
        ]
        prob = cp.Problem(objective, constraints)
        prob.solve(solver='ECOS')
        collect_weight.append(weights.value)
        df_collect.append(betas.drop(betas.columns, axis=1))
        
        # Track performance
        if count % 30 == 0:
            print(date)
        count += 1

    # Create Dataframe
    save_df = pd.concat(df_collect, axis=0)
    # Flatten List
    flattened_array = np.concatenate(collect_weight).flatten()
    flattened_weight = flattened_array.tolist()
    # Add list to column
    save_df['exp_weight'] = flattened_weight
    return save_df

In [ ]:
def mfl(data, stocks_to_use, col_name, exp, min_exp_weights, lambda_reg):
    def objective_cvxpy(weights, betas, min_exp_weights, lambda_reg):
        factor_exposure = 0.0
        for factor, weight in min_exp_weights.items():
            port_beta = cp.matmul(weights, betas[factor])
            factor_exposure += weight * cp.square(port_beta)
        l2_reg = cp.norm(weights, 2)**2
        return factor_exposure + lambda_reg * l2_reg

    collect_weight = []
    df_collect = []
    count = 0
    print(f'Processing {type}...')
    for date, row in stocks_to_use.iterrows():
        stocks = stocks_to_use.loc[stocks_to_use.index == date][col_name][0]
        betas = exp.loc[exp.index.get_level_values('date') == date]
        betas = get_stocks_data(betas, stocks)
        betas = betas.sort_index(level=['permno', 'date'])
        betas = betas.fillna(0)

        # CVXPY setup
        weights = cp.Variable(betas.shape[0])
        total_beta = cp.matmul(betas.to_numpy().T, weights)
            
        objective = cp.Minimize(objective_cvxpy(weights, betas, min_exp_weights, lambda_reg))
        constraints = [
            cp.sum(weights) == 1,
            weights >= 0.001,
            weights <= 0.10
        ]
        prob = cp.Problem(objective, constraints)
        prob.solve(solver='ECOS')
        collect_weight.append(weights.value)
        df_collect.append(betas.drop(betas.columns, axis=1))
        
        if count % 30 == 0:
            print(date)
        count += 1

    # Create Dataframe
    save_df = pd.concat(df_collect, axis=0)
    # Flatten List
    flattened_array = np.concatenate(collect_weight).flatten()
    flattened_weight = flattened_array.tolist()
    # Add list to column
    save_df['exp_weight'] = flattened_weight
    return save_df

In [ ]:
# Expected Return
returns['RET_01_expected'] = returns.groupby('permno')['returns'].shift(1)

In [ ]:
stock = read_stock(get_large(live) / 'permno_to_train_fund.csv')
sb_sector = ModelPrep(live=False, factor_name='factor_sb_sector', group='permno', interval='D', kind='price', stock=stock, div=False, start='2014-01-01', end='2024-01-01', save=False).prep()

In [ ]:
factor_col = ['XLB_RET_01_sector_01_126', 'XLE_RET_01_sector_01_126', 'XLF_RET_01_sector_01_126', 'XLI_RET_01_sector_01_126',
              'XLK_RET_01_sector_01_126', 'XLP_RET_01_sector_01_126', 'XLU_RET_01_sector_01_126', 'XLV_RET_01_sector_01_126',
              'XLY_RET_01_sector_01_126']
exp = sb_sector[factor_col]
min_exp_weights = {
    'XLB_RET_01_sector_01_126': 1,
    'XLE_RET_01_sector_01_126': 1,
    'XLF_RET_01_sector_01_126': 1,
    'XLI_RET_01_sector_01_126': 1,
    'XLK_RET_01_sector_01_126': 1,
    'XLP_RET_01_sector_01_126': 1,
    'XLU_RET_01_sector_01_126': 1,
    'XLV_RET_01_sector_01_126': 1,
    'XLY_RET_01_sector_01_126': 1
}
lambda_reg = 2

In [ ]:
pred['totalStocks'] = pred.apply(lambda row: row['longStocks'] + row['shortStocks'], axis=1)

In [ ]:
long_min_weights = mfl(data=returns, stocks_to_use=pred, col_name='longStocks', exp=exp, min_exp_weights=min_exp_weights, lambda_reg=lambda_reg)

In [ ]:
short_min_weights = mfl(data=returns, stocks_to_use=pred, col_name='shortStocks', exp=exp, min_exp_weights=min_exp_weights, lambda_reg=lambda_reg)

In [ ]:
all_min_weights = mfl(data=returns, stocks_to_use=pred, col_name='totalStocks', exp=exp, min_exp_weights=min_exp_weights, lambda_reg=lambda_reg)

### Calculate Portfolio Return

In [ ]:
def set_exp_weight(data, weight, type):
    data = pd.merge(data, weight, left_index=True, right_index=True, how='left')
    if type == 'long':
        data['total_ret'] = data['returns'] * data['exp_weight'] * 1
    elif type == 'short':
        data['total_ret'] = data['returns'] * data['exp_weight'] * -1
    total_ret = data.groupby('date').total_ret.sum()
    return data, total_ret

In [ ]:
def long_stock(group):
    date = group.index.get_level_values('date')[0]
    list = pred.loc[pred.index == date]['longStocks'][0]
    return get_stocks_data(group, list)

def short_stock(group):
    date = group.index.get_level_values('date')[0]
    list = pred.loc[pred.index == date]['shortStocks'][0]
    return get_stocks_data(group, list)

In [ ]:
long_stocks = returns.groupby('date').apply(long_stock).reset_index(level=0, drop=True)
short_stocks = returns.groupby('date').apply(short_stock).reset_index(level=0, drop=True)

In [ ]:
# long_df, long_ret = set_exp_weight(data=long_stocks[['returns']], weight=long_min_weights, type='long')
long_df, long_ret = set_exp_weight(data=long_stocks[['returns']], weight=all_min_weights, type='long')

In [ ]:
# short_df, short_ret = set_exp_weight(data=short_stocks[['returns']], weight=short_min_weights, type='short')
short_df, short_ret = set_exp_weight(data=short_stocks[['returns']], weight=all_min_weights, type='short')

In [ ]:
total_ret = 0.5 * long_ret + 0.5 * short_ret

In [ ]:
qs.reports.full(total_ret, 'SPY')

# Inverse Volatility

### Calculate Portfolio Return

In [ ]:
vol_returns = returns.copy(deep=True)
vol_returns['accurate_returns'] = vol_returns.groupby('permno')['returns'].shift(1)
vol_returns['vol'] = vol_returns.groupby('permno')['accurate_returns'].transform(lambda x: x.rolling(5).std().shift(1))
vol_returns['inv_vol_weight'] = 1 / vol_returns['vol']

In [ ]:
long_stocks = vol_returns.groupby('date').apply(long_stock).reset_index(level=0, drop=True)
short_stocks = vol_returns.groupby('date').apply(short_stock).reset_index(level=0, drop=True)

In [ ]:
long_stocks['type'] = 1
short_stocks['type'] = -1

In [ ]:
total_stocks = pd.concat([long_stocks, short_stocks], axis=0).sort_index(level=['permno', 'date'])

In [ ]:
# Normalizing Weights
total_stocks['inv_vol_weight'] /= long_stocks.groupby('date')['inv_vol_weight'].transform(lambda x: x.abs().sum())

In [ ]:
total_stocks['total_ret'] = total_stocks['returns'] * total_stocks['inv_vol_weight'] * total_stocks['type']

In [ ]:
total_ret = total_stocks.groupby('date').total_ret.sum()

In [ ]:
qs.reports.full(total_ret, 'SPY')